In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CT_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13
1,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13
2,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13
3,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13
4,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13
1,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13
2,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13
3,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13
4,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1454,1004577101,CT,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3070,7637387710,CT,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
5240,3539977109,CT,"1486 highland ave building 2 unit 1, CHESHIRE, CT",41.508367,-72.910620,NaN
10891,4931747305,CT,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN
11208,9590048601,CT,"143 NEW ENGLAND AVENUE, FAIRFIELD, CT",41.160909,-73.239154,NaN
11568,6030297401,CT,"53 PROSPECT ST APT 100, STAMFORD, CT",41.057584,-73.536661,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('09')] #CT
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/6 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

6


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14417/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1454,1004577101,CT,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,[]
3070,7637387710,CT,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,[]
5240,3539977109,CT,"1486 highland ave building 2 unit 1, CHESHIRE, CT",41.508367,-72.910620,[090093431021000]
10891,4931747305,CT,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,[]
11208,9590048601,CT,"143 NEW ENGLAND AVENUE, FAIRFIELD, CT",41.160909,-73.239154,[090010614001039]
11568,6030297401,CT,"53 PROSPECT ST APT 100, STAMFORD, CT",41.057584,-73.536661,[090010217023000]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_14417/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1454,1004577101,CT,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3070,7637387710,CT,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
5240,3539977109,CT,"1486 highland ave building 2 unit 1, CHESHIRE, CT",41.508367,-72.910620,090093431021000
10891,4931747305,CT,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN
11208,9590048601,CT,"143 NEW ENGLAND AVENUE, FAIRFIELD, CT",41.160909,-73.239154,090010614001039
11568,6030297401,CT,"53 PROSPECT ST APT 100, STAMFORD, CT",41.057584,-73.536661,090010217023000


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1454,1004577101,CT,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3070,7637387710,CT,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
10891,4931747305,CT,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
1454,1004577101,NaN
3070,7637387710,NaN
5240,3539977109,090093431021000
10891,4931747305,NaN
11208,9590048601,090010614001039
11568,6030297401,090010217023000


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/CT_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,183962,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13
1,183963,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13
2,183964,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13
3,183965,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13
4,183966,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,183962,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13,NaN
1,183963,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13,NaN
2,183964,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13,NaN
3,183965,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13,NaN
4,183966,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13,NaN
...,...,...,...,...,...,...,...,...
12877,196839,9618168507,CT,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,9.011705e+13,NaN
12878,196840,9648837000,CT,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,9.003421e+13,NaN
12879,196841,9763078402,CT,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,9.001074e+13,NaN
12880,196842,9854708505,CT,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,9.001011e+13,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,183962,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,9.003502e+13,NaN,90035021002009.0
1,183963,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,9.001110e+13,NaN,90011103022004.0
2,183964,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,9.001022e+13,NaN,90010223003001.0
3,183965,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,9.001110e+13,NaN,90011102023008.0
4,183966,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,9.003502e+13,NaN,90035021001038.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,3903327306,CT,"1 Union Place, Hartford, CT",41.768886,-72.681543,90035021002009.0
1,4505207402,CT,"6 RESEARCH DR, SHELTON, CT",41.274979,-73.134472,90011103022004.0
2,5539227705,CT,"200 First Stamford Place, Stamford, CT",41.044604,-73.549385,90010223003001.0
3,5630057006,CT,"25 CONTROLS DR, SHELTON, CT",41.283920,-73.107990,90011102023008.0
4,8388727000,CT,"242 TRUMBULL ST, HARTFORD, CT",41.768035,-72.675077,90035021001038.0
...,...,...,...,...,...,...
12877,9618168507,CT,"109 Pequotsepos Rd, Mystic, CT",41.362416,-71.946780,90117053001004.0
12878,9648837000,CT,"73 East St, PLAINVILLE, CT",41.669405,-72.861979,90034205004006.0
12879,9763078402,CT,"76 Knowlton St, Bridgeport, CT",41.184879,-73.187417,90010739003009.0
12880,9854708505,CT,"15 E Putnam Ave, Greenwich, CT",41.031819,-73.626677,90010106002002.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
1454,1004577101,CT,"120 Long Ridge Road 3 North, STAMFORD, CT",41.195893,-73.620851,NaN
3070,7637387710,CT,"5200 Rte 42, Turnersville, CT",39.765211,-75.048383,NaN
10891,4931747305,CT,"180 Varick Street, SALISBURY, CT",40.727398,-74.005178,NaN


Convert to File

In [19]:
fips_CT = fips_merge_drop

In [20]:
fips_CT.to_csv("../../../data/state_data/geo/geo_fips/23/CT_fips_scraped.csv") 